## Determine if the input Audio are Cough or Speech

### Model trained by Abil

In [1]:
import librosa as lb
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
import sys

#### Load Audio Sample

In server, the audio upload path is */usr/src/app/public/uploads/batuk/*

But here using sample wav

In [2]:
#sound, sample_rate = lb.load("out.wav")
sound, sample_rate = lb.load("covid_dataset/wav_normalized/0aa64689-c48a-421d-b353-c3496bad51ed.wav")

#### Get all audio parameter

In [3]:
stft = np.abs(lb.stft(sound))  
mfccs = np.mean(lb.feature.mfcc(y=sound, sr=sample_rate, n_mfcc=40),axis=1)
chroma = np.mean(lb.feature.chroma_stft(S=stft, sr=sample_rate),axis=1)
mel = np.mean(lb.feature.melspectrogram(sound, sr=sample_rate),axis=1)
contrast = np.mean(lb.feature.spectral_contrast(S=stft, sr=sample_rate),axis=1)
tonnetz = np.mean(lb.feature.tonnetz(y=lb.effects.harmonic(sound), sr=sample_rate),axis=1)
concat = np.concatenate((mfccs,chroma,mel,contrast,tonnetz))

#### Pack all parameters in parameter image/array

In [4]:
images_tes = np.array(concat)
images_tes = np.reshape(images_tes, (1, images_tes.shape[0], 1))

#### Load Pre-Trained Neural Network Model

In [5]:
model = keras.models.load_model("./saved-model/cough_01.h5")

2022-01-29 10:34:52.298965: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-29 10:34:52.299943: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


#### Get prediction based on Pre-Trained Model

In [6]:
preds = model.predict(images_tes)
classpreds = np.argmax(preds, axis=1)

print(classpreds[0])

0


2022-01-29 10:34:52.772507: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


#### Value 0 means Cough, value 1 means Speech

In [7]:
if classpreds[0] == 0:
    print("Cough")
else:
    print("Non-Cough")

Cough
